<a href="https://colab.research.google.com/github/JonRivera/DS-Unit-1-Sprint-2-Statistics/blob/master/module2/LS_DS_122_Chi2_Tests_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment - Practice Chi^2 Tests

Use the following dataset relating to math scores of students in two different Portugese schools:

<https://archive.ics.uci.edu/ml/datasets/Student+Performance>

### 1) Load the dataset specifically relating to math scores as a new dataframe.

There are two datasets in the `student.zip` file, make sure you use `student-mat.csv`.


In [1]:
# YOUR WORK HERE
import numpy as np
from scipy import stats
import pandas as pd
from scipy.stats import chisquare

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip

--2020-06-09 20:52:47--  https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20478 (20K) [application/x-httpd-php]
Saving to: ‘student.zip.1’

student.zip.1       100%[===================>]  20.00K  --.-KB/s    in 0.1s    

2020-06-09 20:52:48 (146 KB/s) - ‘student.zip.1’ saved [20478/20478]



In [2]:
!unzip student.zip


Archive:  student.zip
replace student-mat.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: student-mat.csv         
replace student-por.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: student-por.csv         
replace student-merge.R? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: student-merge.R         
replace student.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: student.txt             


In [3]:
df = pd.read_csv('student-mat.csv', sep=';')
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


### 2) Use Chi^2 tests and `stats.chi2_contingency()` to identify:
 - Two pairs of variables that are dependent (are associated with one another).
 - Two pairs of variables that are independent (have no significant relationship).

Does it make intuitive sense why the variables in these pairs might or might not show a relationship to one another? 


In [29]:
# YOUR WORK HERE
#pairs of variables are possibly dependent
contingency1 = pd.crosstab(df['studytime'], df['freetime'])
contingency2 = pd.crosstab(df['traveltime'],df['absences'])

print(contingency1,contingency2)

freetime    1   2   3   4   5
studytime                    
1           2  19  35  29  20
2          11  28  82  61  16
3           3  11  29  21   1
4           3   6  11   4   3 absences    0   1   2   3   4   5   6   7   ...  26  28  30  38  40  54  56  75
traveltime                                  ...                                
1           74   2  40   5  37   4  16   5  ...   0   1   0   0   1   1   1   0
2           32   1  20   2  13   1  10   1  ...   1   0   1   1   0   0   0   1
3            6   0   4   1   3   0   3   1  ...   0   0   0   0   0   0   0   0
4            3   0   1   0   0   0   2   0  ...   0   0   0   0   0   0   0   0

[4 rows x 34 columns]


Null Hypothesis: Study time and free time variables are independent.
Alternative: The two variables are dependent and there is a relationship associated between them.

In [12]:
chi2, p_value, dof, expected = stats.chi2_contingency(contingency1)
print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table", expected)

chi2 statistic 23.53045426913059
p value 0.023545617681757027
degrees of freedom 12
expected frequencies table [[ 5.05063291 17.01265823 41.73417722 30.56962025 10.63291139]
 [ 9.52405063 32.08101266 78.69873418 57.64556962 20.05063291]
 [ 3.12658228 10.53164557 25.83544304 18.92405063  6.58227848]
 [ 1.29873418  4.37468354 10.73164557  7.86075949  2.73417722]]


Conclusion:  Given the p value of 0.023 study time , this suggest the variables are not independednt and instead might be depenedent. 

Intuition: Initially, I my gut feeling was that study time and free time are correlated. If a student studies more, logically, he might have less time available for free time.

Null Hypothesis: Absences and Travel time variables are  independent.
Alternative: The two variables are dependednt and there is a relationship associated between them.


In [0]:
chi2, p_value, dof, expected = stats.chi2_contingency(contingency2)
print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table", expected)

chi2 statistic 61.11558335253091
p value 0.9990069240658936
degrees of freedom 99
expected frequencies table [[7.48227848e+01 1.95189873e+00 4.22911392e+01 5.20506329e+00
  3.44835443e+01 3.25316456e+00 2.01696203e+01 4.55443038e+00
  1.43139241e+01 1.95189873e+00 1.10607595e+01 1.95189873e+00
  7.80759494e+00 1.95189873e+00 7.80759494e+00 1.95189873e+00
  4.55443038e+00 6.50632911e-01 3.25316456e+00 6.50632911e-01
  2.60253165e+00 6.50632911e-01 1.95189873e+00 6.50632911e-01
  6.50632911e-01 6.50632911e-01 6.50632911e-01 6.50632911e-01
  6.50632911e-01 6.50632911e-01 6.50632911e-01 6.50632911e-01
  6.50632911e-01 6.50632911e-01]
 [3.11518987e+01 8.12658228e-01 1.76075949e+01 2.16708861e+00
  1.43569620e+01 1.35443038e+00 8.39746835e+00 1.89620253e+00
  5.95949367e+00 8.12658228e-01 4.60506329e+00 8.12658228e-01
  3.25063291e+00 8.12658228e-01 3.25063291e+00 8.12658228e-01
  1.89620253e+00 2.70886076e-01 1.35443038e+00 2.70886076e-01
  1.08354430e+00 2.70886076e-01 8.12658228e-01 2.708

Conclusion: Given a p vaue of .99 there is evidence to suggest that travel time and absences might be independent.

Intution: its not necessarily clear how travel time might affect absences. It really depends on wheather a student has reliable transportation, wake up time and other factors as well might come to play.

In [0]:
chi2, p_value, dof, expected = stats.chi2_contingency(contingency2)
print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table", expected)

In [0]:
#pairs of variables that are possibly independent
contingency3 = pd.crosstab(df['Fedu'],df['failures'])
contingency4 = pd.crosstab(df['Medu'],df['failures'])

In [11]:
print(contingency3,contingency4)

failures   0   1  2   3
Fedu                   
0          2   0  0   0
1         51  16  3  12
2         88  17  7   3
3         85  10  4   1
4         86   7  3   0 failures    0   1  2  3
Medu                   
0           2   0  0  1
1          35  15  2  7
2          81  14  5  3
3          73  14  7  5
4         121   7  3  0


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Null Hypothesis: Father education and failures variables are independent. Alternative: The two variables are dependent and there is a relationship associated between them.


In [21]:
chi2, p_value, dof, expected = stats.chi2_contingency(contingency3)
print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table", expected)

chi2 statistic 42.277686186072714
p value 2.990535490026923e-05
degrees of freedom 12
expected frequencies table [[1.57974684e+00 2.53164557e-01 8.60759494e-02 8.10126582e-02]
 [6.47696203e+01 1.03797468e+01 3.52911392e+00 3.32151899e+00]
 [9.08354430e+01 1.45569620e+01 4.94936709e+00 4.65822785e+00]
 [7.89873418e+01 1.26582278e+01 4.30379747e+00 4.05063291e+00]
 [7.58278481e+01 1.21518987e+01 4.13164557e+00 3.88860759e+00]]


Conclusion: Given a p value of 2.990535490026923e-05 and statistic of 42.27, I reject the null hypothesis father education and failures are independent. Instead, I suggest fathers education might be dependent to a students failure.

Intuition: It makes sense in general for at least on parent(father) to contribute to students education lack of failure. Might have to do with better habbits/goals motivated by parent.

Null Hypothesis: Mother education and failures variables are independent. Alternative: The two variables are dependent and there is a relationship associated between them.

In [22]:
chi2, p_value, dof, expected = stats.chi2_contingency(contingency4)
print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table", expected)

chi2 statistic 44.45276808268604
p value 1.2783690573466937e-05
degrees of freedom 12
expected frequencies table [[  2.36962025   0.37974684   0.12911392   0.12151899]
 [ 46.60253165   7.46835443   2.53924051   2.38987342]
 [ 81.35696203  13.03797468   4.43291139   4.1721519 ]
 [ 78.19746835  12.53164557   4.26075949   4.01012658]
 [103.47341772  16.58227848   5.63797468   5.30632911]]


Conclusion: Given a p value of 1.2783690573466937e-05 and statistic of 44.45, I reject the null hypothesis mother education and failures are independent. Instead, I suggest mothers education might be dependent to a students failure.

Intuition: It makes sense in general for at least on parent to contribute to students education lack of failure. Might have to do with better habbits/goals motivated by parent.

### 3) Use NumPy to perform your own chi^2 test "from scratch" 

Pick any of the chi^2 tests that you ran in part 2 and try them on your own. You should get the same results that Scipy got for all four values returned from `chi2_contingency()`

In [0]:
# YOUR WORK HERE
# Using NumPy to perform Chi^2 from scratch


In [4]:
df['traveltime'].value_counts()

1    257
2    107
3     23
4      8
Name: traveltime, dtype: int64

In [5]:
df['absences'].value_counts()

0     115
2      65
4      53
6      31
8      22
10     17
14     12
12     12
3       8
7       7
16      7
18      5
5       5
20      4
22      3
13      3
1       3
9       3
11      3
15      3
23      1
24      1
21      1
25      1
56      1
26      1
28      1
30      1
17      1
38      1
40      1
54      1
19      1
75      1
Name: absences, dtype: int64

In [0]:
observed = pd.crosstab(df['traveltime'],df['absences'])

In [7]:
observed_with_margins = pd.crosstab(df['traveltime'], df['absences'], margins=True)
observed_with_margins

absences,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,28,30,38,40,54,56,75,All
traveltime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,74,2,40,5,37,4,16,5,13,3,12,3,9,2,8,3,6,1,4,1,2,1,1,1,0,0,0,1,0,0,1,1,1,0,257
2,32,1,20,2,13,1,10,1,6,0,4,0,3,1,3,0,1,0,1,0,0,0,2,0,1,1,1,0,1,1,0,0,0,1,107
3,6,0,4,1,3,0,3,1,2,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,23
4,3,0,1,0,0,0,2,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
All,115,3,65,8,53,5,31,7,22,3,17,3,12,3,12,3,7,1,5,1,4,1,3,1,1,1,1,1,1,1,1,1,1,1,395


In [8]:
#Making an expeted value table corresponding to observed values
observed = observed.values
observed

array([[74,  2, 40,  5, 37,  4, 16,  5, 13,  3, 12,  3,  9,  2,  8,  3,
         6,  1,  4,  1,  2,  1,  1,  1,  0,  0,  0,  1,  0,  0,  1,  1,
         1,  0],
       [32,  1, 20,  2, 13,  1, 10,  1,  6,  0,  4,  0,  3,  1,  3,  0,
         1,  0,  1,  0,  0,  0,  2,  0,  1,  1,  1,  0,  1,  1,  0,  0,
         0,  1],
       [ 6,  0,  4,  1,  3,  0,  3,  1,  2,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 3,  0,  1,  0,  0,  0,  2,  0,  1,  0,  0,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0]])

In [9]:
#Expected Value Calculation
#First obtain all row totals
row_totals = df['traveltime'].value_counts().sort_index().values
#column totals
col_totals = df['absences'].value_counts().sort_index().values
print(row_totals)
print(col_totals)

[257 107  23   8]
[115   3  65   8  53   5  31   7  22   3  17   3  12   3  12   3   7   1
   5   1   4   1   3   1   1   1   1   1   1   1   1   1   1   1]


In [0]:
#sample size = # of rows
sample_size = df.shape[0]

In [10]:
expected = []
total = 395
for i in range(len(row_totals)):
  expected_row = []
  for column in col_totals:
    expected_val = column * row_totals[i]/total
    expected_row.append(expected_val)
  expected.append(expected_row)
expected = np.array(expected)


print(expected.shape)
print(expected)

(4, 34)
[[7.48227848e+01 1.95189873e+00 4.22911392e+01 5.20506329e+00
  3.44835443e+01 3.25316456e+00 2.01696203e+01 4.55443038e+00
  1.43139241e+01 1.95189873e+00 1.10607595e+01 1.95189873e+00
  7.80759494e+00 1.95189873e+00 7.80759494e+00 1.95189873e+00
  4.55443038e+00 6.50632911e-01 3.25316456e+00 6.50632911e-01
  2.60253165e+00 6.50632911e-01 1.95189873e+00 6.50632911e-01
  6.50632911e-01 6.50632911e-01 6.50632911e-01 6.50632911e-01
  6.50632911e-01 6.50632911e-01 6.50632911e-01 6.50632911e-01
  6.50632911e-01 6.50632911e-01]
 [3.11518987e+01 8.12658228e-01 1.76075949e+01 2.16708861e+00
  1.43569620e+01 1.35443038e+00 8.39746835e+00 1.89620253e+00
  5.95949367e+00 8.12658228e-01 4.60506329e+00 8.12658228e-01
  3.25063291e+00 8.12658228e-01 3.25063291e+00 8.12658228e-01
  1.89620253e+00 2.70886076e-01 1.35443038e+00 2.70886076e-01
  1.08354430e+00 2.70886076e-01 8.12658228e-01 2.70886076e-01
  2.70886076e-01 2.70886076e-01 2.70886076e-01 2.70886076e-01
  2.70886076e-01 2.70886076e-

In [11]:
print(observed)

[[74  2 40  5 37  4 16  5 13  3 12  3  9  2  8  3  6  1  4  1  2  1  1  1
   0  0  0  1  0  0  1  1  1  0]
 [32  1 20  2 13  1 10  1  6  0  4  0  3  1  3  0  1  0  1  0  0  0  2  0
   1  1  1  0  1  1  0  0  0  1]
 [ 6  0  4  1  3  0  3  1  2  0  1  0  0  0  0  0  0  0  0  0  2  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 3  0  1  0  0  0  2  0  1  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]]


In [13]:
#compute chi^2 static
((observed - expected)**2/expected).sum()

61.11558335253091

In [20]:
observed.shape

(4, 34)

In [23]:
#degrees of freedom
#2-sample (test for independence), DOF = (# rows_crosstab-1)*(# cols_crosstab-1)

dof = (4-1)*(34-1)
print(dof)

99


In [17]:
#verify chi static w/ scipy
#SciPy confirms calculated value of 61.115
contingency2 = pd.crosstab(df['traveltime'],df['absences'])
chi2, p_value, dof, expected = stats.chi2_contingency(contingency2)
chi2, p_value, dof, expected = stats.chi2_contingency(contingency2)
print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table", expected)


chi2 statistic 61.11558335253091
p value 0.9990069240658936
degrees of freedom 99
expected frequencies table [[7.48227848e+01 1.95189873e+00 4.22911392e+01 5.20506329e+00
  3.44835443e+01 3.25316456e+00 2.01696203e+01 4.55443038e+00
  1.43139241e+01 1.95189873e+00 1.10607595e+01 1.95189873e+00
  7.80759494e+00 1.95189873e+00 7.80759494e+00 1.95189873e+00
  4.55443038e+00 6.50632911e-01 3.25316456e+00 6.50632911e-01
  2.60253165e+00 6.50632911e-01 1.95189873e+00 6.50632911e-01
  6.50632911e-01 6.50632911e-01 6.50632911e-01 6.50632911e-01
  6.50632911e-01 6.50632911e-01 6.50632911e-01 6.50632911e-01
  6.50632911e-01 6.50632911e-01]
 [3.11518987e+01 8.12658228e-01 1.76075949e+01 2.16708861e+00
  1.43569620e+01 1.35443038e+00 8.39746835e+00 1.89620253e+00
  5.95949367e+00 8.12658228e-01 4.60506329e+00 8.12658228e-01
  3.25063291e+00 8.12658228e-01 3.25063291e+00 8.12658228e-01
  1.89620253e+00 2.70886076e-01 1.35443038e+00 2.70886076e-01
  1.08354430e+00 2.70886076e-01 8.12658228e-01 2.708

In [0]:
#using https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html compute p value
#right tail probability is(p-value) is .999


## Stretch goals:

### 1. Refactor your code so it is elegant, readable, and holds reusable code in functions.

In [19]:
# YOUR WORK HERE


(4, 34)



### 2. Check For Understanding - Study and write your own explanations/definitions for these topics:

- What is a sample "estimate" in statistics?

- What are hypothesis test? How are they useful?

- What is a "null hypothesis?"

- What is a p-value and what does it represent?

- What does it mean for something to be "statistically significant?"

- What is a test statistic and how does it relate to a p-value?

- What are "degrees of freedom" and how are they calculated in a 1-sample, 2-sample, and chi^2 test for independence?

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)